# Vorbereitung
Import und einlesen der Daten, Überprüfen der Form (Spalten, Zeilen) der Datensätze

In [47]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
import plotly.express as px
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

import plotly.graph_objs as go

In [48]:
X_train = pd.read_csv('Xtrain_tvshows_featured.csv')
X_test = pd.read_csv('Xtest_tvshows_featured.csv')
y_train = pd.read_csv('ytrain_tvshows.csv')
y_test = pd.read_csv('ytest_tvshows.csv')
print("Shape of X Train: {}".format(X_train.shape))
print("Shape of X Test: {}".format(X_test.shape))
print("Shape of y Train: {}".format(y_train.shape))
print("Shape of y Test: {}".format(y_test.shape))

Shape of X Train: (4294, 7)
Shape of X Test: (1074, 7)
Shape of y Train: (4294, 1)
Shape of y Test: (1074, 1)


# Optimizing GBM-Model

training set: 0.8320912901723335
test set: 0.8100558659217877

In [75]:
# defining the parameters
parameters= {'learning_rate':[0.19,0.2],
             'n_estimators':[130,150,170],
             'max_depth':[4,5],
             'min_samples_leaf':[1]}
#parameters= {'learning_rate':[0.1,0.01],
#             'n_estimators':[100,1000],
#             'max_depth':[2,3,4],
#             'min_samples_leaf':[200]}
# defining cross-validation method
#cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# defining the model
model = GradientBoostingClassifier()
# defining the grid serach (full grid here)
#Define the scoring
clf=GridSearchCV(model,parameters,scoring='recall',cv=5,n_jobs=-1) # with n_jobs=-1 we run the computation in parallel
clf.fit(X=X_train, y=y_train)
# getting best parameters
print(clf.best_params_)

/Users/silyr/venv/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/silyr/venv/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/silyr/venv/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/silyr/venv/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

{'learning_rate': 0.19, 'max_depth': 4, 'min_samples_leaf': 1, 'n_estimators': 170}


In [70]:
# Get the best parameters found by the random search
best_params = clf.best_params_

# Create a dictionary of hyperparameter names and their values
hyperparams = {
    'n_estimators': best_params['n_estimators'],
    'learning_rate': best_params['learning_rate'],
    'max_depth': best_params['max_depth'],
    #'min_samples_split': best_params['min_samples_split'],
    'min_samples_leaf': best_params['min_samples_leaf'],
    #'max_features': best_params['max_features']
}

# Create a bar chart of the hyperparameters
fig = go.Figure(data=[go.Bar(
    x=list(hyperparams.keys()),
    y=list(hyperparams.values()),
    marker=dict(
        color=['rgba(255, 0, 0, 0.7)' if key in best_params else 'rgba(0, 0, 255, 0.7)' for key in hyperparams.keys()]
    )
)])
# Set the chart title and axis labels
fig.update_layout(
    title='Best Hyperparameters',
    xaxis_title='Hyperparameter',
    yaxis_title='Value'
)

# Display the chart
fig.show()

In [71]:
# Get the best CV model
best_model = clf.best_estimator_

# Make a prediction for the test set
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

# Print the predicted values
print(y_pred_test)

[1 1 1 ... 0 0 1]


In [72]:
print("Confusion matrix of the training set: {}".format(metrics.confusion_matrix(y_train,y_pred_train)))
print("Confusion matrix of the test set: {}".format(metrics.confusion_matrix(y_test,y_pred_test)))

Confusion matrix of the training set: [[2345  228]
 [ 313 1408]]
Confusion matrix of the test set: [[575  93]
 [112 294]]


In [73]:
print("Accuracy Score for the training set: {}".format(metrics.accuracy_score(y_train, y_pred_train)))
print("Accuracy Score for the test set: {}".format(metrics.accuracy_score(y_test, y_pred_test)))

Accuracy Score for the training set: 0.8740102468560782
Accuracy Score for the test set: 0.8091247672253259
